In [1]:
import xarray as xr
from dask.diagnostics import ProgressBar
import pandas as pd
from shapely.geometry import Polygon, MultiPolygon, Point
import geopandas as gpd
import numpy as np
import cartopy.crs as ccrs
import cartopy.feature as cfeature
from tqdm import tqdm  # Optional: For progress tracking

import glob



In [2]:

import unicodedata

# Function to normalize the names
def normalize_municipality_name(name):
    if type(name) == float:
        return name
    else:
        # Normalize the string (remove diacritical marks)
        name_without_diacritics = unicodedata.normalize('NFKD', name).encode('ascii', 'ignore').decode('ascii')
        # Convert to lowercase
        return name_without_diacritics.lower()

mun_gdf = gpd.read_file(r'/Volumes/Dhruv_External_Disk/portugal_borders/concelhos-shapefile/concelhos.shp')
mun_metadata  = pd.read_excel(r'/Volumes/Dhruv_External_Disk/portugal_borders/concelhos-metadata.xlsx', dtype={'dicofre' : 'string'})
mun_metadata = mun_metadata[['dicofre','designacao']]

mun_gdf = mun_gdf.merge(mun_metadata, left_on='CCA_2', right_on='dicofre').drop(columns='NAME_2')
mun_gdf = mun_gdf.rename(columns={'designacao': 'NAME_2'})
mun_gdf['concelho'] = mun_gdf['NAME_2'].map(normalize_municipality_name)
mun_gdf = mun_gdf[~mun_gdf['NAME_1'].isin(['Azores', 'Madeira'])]
west_pt, south_pt, east_pt, north_pt = mun_gdf.total_bounds

In [3]:
nuts = gpd.read_file(r'/Volumes/Dhruv_External_Disk/portugal_borders/NUTS_RG_20M_2024_4326.shp/NUTS_RG_20M_2024_4326.shp')
nuts['NUTS_NAME'] = nuts['NUTS_NAME'].map(normalize_municipality_name)

nuts_iberia = nuts[(nuts['CNTR_CODE'].isin(['PT', 'ES'])) &  (nuts['LEVL_CODE'] == 2) & (~nuts['NUTS_NAME'].isin(['canarias', 'regiao autonoma dos acores', 'regiao autonoma da madeira', 'illes balears']))]
west, south, east, north = nuts_iberia.total_bounds

In [4]:
IBERIA_BBOX = {
    "min_lat": south,  # Minimum latitude
    "max_lat": north,  # Maximum latitude
    "min_lon": west,   # Minimum longitude
    "max_lon": east    # Maximum longitude
}
IBERIA_BBOX

{'min_lat': 35.271704476000025,
 'max_lat': 43.735114829000054,
 'min_lon': -9.490672837999966,
 'max_lon': 3.25861242000002}

In [ ]:
# import math

# def round_bbox(north, south, west, east):
#     """
#     Round each coordinate to the nearest 0.1° according to:
#       north, east -> round up
#       south, west -> round down
#     Returns a dict ready to drop into your CDS request.
#     """
#     north_1dp = math.ceil(north * 10) / 10       # up
#     south_1dp = math.floor(south * 10) / 10      # down
#     east_1dp  = math.ceil(east  * 10) / 10       # up
#     west_1dp  = math.floor(west  * 10) / 10      # down

#     return {
#         "min_lat": south_1dp,
#         "max_lat": north_1dp,
#         "min_lon": west_1dp,
#         "max_lon": east_1dp
#     }

# # --- example ---

# IBERIA_BBOX = round_bbox(north, south, west, east)
# print(IBERIA_BBOX)


{'min_lat': 35.2, 'max_lat': 43.8, 'min_lon': -9.5, 'max_lon': 3.3}


### Daily Temperature Means

In [5]:
import cdsapi, pathlib, time, concurrent.futures

# ---- edit your bounding box here ----


OUTDIR = pathlib.Path("/Volumes/Dhruv_External_Disk/climate_data/temperature/daily_data/daily_means")
OUTDIR.mkdir(parents=True, exist_ok=True)

def _day_list():
    return [f"{d:02d}" for d in range(1, 32)]
def _month_list():
    return [f"{m:02d}" for m in range(1, 13)]
def _hour_list():
    return [f"{h:02d}:00" for h in range(24)]


In [10]:
def download_daily_mean(year):
    req = {
        "variable": ["2m_temperature"],
        "year": [str(year)],
        "month": _month_list(),
        "day": _day_list(),
        "daily_statistic": "daily_mean",
        "time_zone": "utc+00:00",
        "frequency": "1_hourly",
        "area": [north, west, south, 
        east],
        "format": "grib"
    }
    target = OUTDIR / f"tmean_daily_{year}.grib"
    if target.exists():
        print(f"✓ {target.name} exists - skip"); return
    cdsapi.Client().retrieve("derived-era5-land-daily-statistics", req, str(target))

for yr in range(2020, 2024): #was 1991
    download_daily_mean(yr)


2025-07-14 15:41:54,589 INFO [2024-09-26T00:00:00] Watch our [Forum](https://forum.ecmwf.int/) for Announcements, news and other discussed topics.
2025-07-14 15:41:55,076 INFO Request ID is 3c83c91b-8775-4b28-8e56-5570b326aacc
2025-07-14 15:41:55,167 INFO status has been updated to accepted
2025-07-14 15:42:03,829 INFO status has been updated to running
2025-07-14 16:00:15,912 INFO status has been updated to successful


292f476247a6a62ba46fb8b6f24bf85c.nc:   0%|          | 0.00/6.25M [00:00<?, ?B/s]

2025-07-14 16:00:18,187 INFO [2024-09-26T00:00:00] Watch our [Forum](https://forum.ecmwf.int/) for Announcements, news and other discussed topics.
2025-07-14 16:00:18,558 INFO Request ID is 25c4ef82-a811-4729-b61d-58cabd20f5fe
2025-07-14 16:00:18,654 INFO status has been updated to accepted
2025-07-14 16:00:39,906 INFO status has been updated to running
2025-07-14 16:22:39,654 INFO status has been updated to successful


a1b21fb8436c4ed2b49b10bd942ea4f0.nc:   0%|          | 0.00/6.24M [00:00<?, ?B/s]

2025-07-14 16:22:41,707 INFO [2024-09-26T00:00:00] Watch our [Forum](https://forum.ecmwf.int/) for Announcements, news and other discussed topics.
2025-07-14 16:22:42,273 INFO Request ID is d6c5a592-48f4-45fe-a229-8e0b6c9cf331
2025-07-14 16:22:42,347 INFO status has been updated to accepted
2025-07-14 16:23:32,763 INFO status has been updated to running
2025-07-14 16:41:03,729 INFO status has been updated to successful


1c3c24260d90b533f373fccf6ad96775.nc:   0%|          | 0.00/6.24M [00:00<?, ?B/s]

2025-07-14 16:41:15,416 INFO [2024-09-26T00:00:00] Watch our [Forum](https://forum.ecmwf.int/) for Announcements, news and other discussed topics.
2025-07-14 16:41:15,645 INFO Request ID is 40c332f3-b2fe-4ff0-82dc-80ad8a0ab6e1
2025-07-14 16:41:15,969 INFO status has been updated to accepted
2025-07-14 16:41:37,270 INFO status has been updated to running
2025-07-14 17:03:37,318 INFO status has been updated to successful


4df01bde22af53bd4c7466806c18d107.nc:   0%|          | 0.00/6.24M [00:00<?, ?B/s]

In [7]:
def download_daily_mean(year):
    req = {
        "variable": ["2m_temperature"],
        "year": [str(year)],
        "month": _month_list(),
        "day": _day_list(),
        "daily_statistic": "daily_mean",
        "time_zone": "utc+00:00",
        "frequency": "1_hourly",
        "area": [IBERIA_BBOX['max_lat'], IBERIA_BBOX['min_lon'], IBERIA_BBOX['min_lat'], IBERIA_BBOX['max_lon']],
        "format": "grib"
    }
    target = OUTDIR / f"tmean_daily_{year}.grib"
    if target.exists():
        print(f"✓ {target.name} exists - skip"); return
    cdsapi.Client().retrieve("derived-era5-land-daily-statistics", req, str(target))

for yr in range(1959, 1961):
    download_daily_mean(yr)


2025-07-17 15:16:03,922 INFO [2024-09-26T00:00:00] Watch our [Forum](https://forum.ecmwf.int/) for Announcements, news and other discussed topics.
2025-07-17 15:16:04,334 INFO Request ID is cbcf7116-994d-436d-9d88-0eae484e9ca3
2025-07-17 15:16:04,414 INFO status has been updated to accepted
2025-07-17 15:16:54,672 INFO status has been updated to running
2025-07-17 15:38:25,727 INFO status has been updated to successful


5ae1c00f2811313214917ebb1b1194ca.nc:   0%|          | 0.00/6.23M [00:00<?, ?B/s]

2025-07-17 15:38:27,886 INFO [2024-09-26T00:00:00] Watch our [Forum](https://forum.ecmwf.int/) for Announcements, news and other discussed topics.
2025-07-17 15:38:28,100 INFO Request ID is b6c50458-53db-476d-bb7d-c47a7169ac23
2025-07-17 15:38:28,204 INFO status has been updated to accepted
2025-07-17 15:39:00,987 INFO status has been updated to running
2025-07-17 16:00:49,991 INFO status has been updated to successful


6b6ea35a5f0041e3bf51442d56a3652b.nc:   0%|          | 0.00/6.25M [00:00<?, ?B/s]

### Daily Temperature Max

In [5]:
import cdsapi, pathlib, time, concurrent.futures

# ---- edit your bounding box here ----


OUTDIR = pathlib.Path("/Volumes/Dhruv_External_Disk/climate_data/temperature/daily_data/daily_max")
OUTDIR.mkdir(parents=True, exist_ok=True)

def _day_list():
    return [f"{d:02d}" for d in range(1, 32)]
def _month_list():
    return [f"{m:02d}" for m in range(1, 13)]
def _hour_list():
    return [f"{h:02d}:00" for h in range(24)]


In [7]:
def download_daily_mean(year):
    req = {
        "variable": ["2m_temperature"],
        "year": [str(year)],
        "month": _month_list(),
        "day": _day_list(),
        "daily_statistic": "daily_maximum",
        "time_zone": "utc+00:00",
        "frequency": "1_hourly",
        "area": [north, west, south, 
        east],
        "format": "grib"
    }
    target = OUTDIR / f"tmean_daily_{year}.grib"
    if target.exists():
        print(f"✓ {target.name} exists - skip"); return
    cdsapi.Client().retrieve("derived-era5-land-daily-statistics", req, str(target))

for yr in [1981, 1982, 1974, 1960, 1953]: #was 1991
    download_daily_mean(yr)


2025-09-05 10:55:24,697 INFO [2024-09-26T00:00:00] Watch our [Forum](https://forum.ecmwf.int/) for Announcements, news and other discussed topics.
2025-09-05 10:55:24,964 INFO Request ID is 4299a29b-5549-47a2-a0c2-61606392333d
2025-09-05 10:55:25,061 INFO status has been updated to accepted
2025-09-05 10:55:38,868 INFO status has been updated to running
2025-09-05 11:15:46,755 INFO status has been updated to successful


879ead4c4779094deb08eba5e94cb230.nc:   0%|          | 0.00/5.44M [00:00<?, ?B/s]

2025-09-05 11:15:49,140 INFO [2024-09-26T00:00:00] Watch our [Forum](https://forum.ecmwf.int/) for Announcements, news and other discussed topics.
2025-09-05 11:15:49,403 INFO Request ID is a2ba8758-16b1-44dc-9319-2a1f2e57c7a0
2025-09-05 11:15:49,485 INFO status has been updated to accepted
2025-09-05 11:16:04,806 INFO status has been updated to running
2025-09-05 11:36:13,561 INFO status has been updated to successful


1dd812ddbd3bf1b5cf3b98467a5a9a9a.nc:   0%|          | 0.00/5.43M [00:00<?, ?B/s]

2025-09-05 11:36:14,912 INFO [2024-09-26T00:00:00] Watch our [Forum](https://forum.ecmwf.int/) for Announcements, news and other discussed topics.
2025-09-05 11:36:15,142 INFO Request ID is db00b656-6e0c-417a-8866-43105b137fdc
2025-09-05 11:36:15,231 INFO status has been updated to accepted
2025-09-05 11:36:23,758 INFO status has been updated to running
2025-09-05 11:54:36,386 INFO status has been updated to successful


643b8babd95795c7abafcd82c7f6d356.nc:   0%|          | 0.00/5.43M [00:00<?, ?B/s]

2025-09-05 11:54:37,681 INFO [2024-09-26T00:00:00] Watch our [Forum](https://forum.ecmwf.int/) for Announcements, news and other discussed topics.
2025-09-05 11:54:37,883 INFO Request ID is 8aba7c9c-4353-41a6-b334-f5f4252423b6
2025-09-05 11:54:37,944 INFO status has been updated to accepted
2025-09-05 11:55:12,908 INFO status has been updated to running
2025-09-05 12:24:58,661 INFO status has been updated to successful


26c8093415953cead1045e6211e1322c.nc:   0%|          | 0.00/5.46M [00:00<?, ?B/s]

2025-09-05 12:25:00,133 INFO [2024-09-26T00:00:00] Watch our [Forum](https://forum.ecmwf.int/) for Announcements, news and other discussed topics.
2025-09-05 12:25:00,315 INFO Request ID is 703bfd4f-b9cd-4f49-b960-54b046c8a740
2025-09-05 12:25:00,760 INFO status has been updated to accepted
Recovering from connection error [('Connection aborted.', RemoteDisconnected('Remote end closed connection without response'))], attemps 1 of 500
Retrying in 120 seconds
2025-09-05 13:15:56,745 INFO status has been updated to successful


da49abc9378601a5ce05861a965d1082.nc:   0%|          | 0.00/5.44M [00:00<?, ?B/s]

In [10]:
def download_daily_mean(year):
    req = {
        "variable": ["2m_temperature"],
        "year": [str(year)],
        "month": _month_list(),
        "day": _day_list(),
        "daily_statistic": "daily_maximum",
        "time_zone": "utc+00:00",
        "frequency": "1_hourly",
        "area": [north, west, south, 
        east],
        "format": "grib"
    }
    target = OUTDIR / f"tmean_daily_{year}.grib"
    if target.exists():
        print(f"✓ {target.name} exists - skip"); return
    cdsapi.Client().retrieve("derived-era5-land-daily-statistics", req, str(target))

for yr in range(2004, 2010): #was 1991
    download_daily_mean(yr)


2025-09-09 12:00:24,615 INFO [2024-09-26T00:00:00] Watch our [Forum](https://forum.ecmwf.int/) for Announcements, news and other discussed topics.
2025-09-09 12:00:24,834 INFO Request ID is de365a12-091b-489a-acbe-9523903e4a10
2025-09-09 12:00:24,909 INFO status has been updated to accepted
2025-09-09 12:00:38,634 INFO status has been updated to running
2025-09-09 12:20:46,186 INFO status has been updated to successful


d4116d387399772111f5c59ff51403f2.nc:   0%|          | 0.00/5.46M [00:00<?, ?B/s]

2025-09-09 12:20:47,458 INFO [2024-09-26T00:00:00] Watch our [Forum](https://forum.ecmwf.int/) for Announcements, news and other discussed topics.
2025-09-09 12:20:47,695 INFO Request ID is eaeeb1fa-1aa1-4db5-a372-5387452b32b6
2025-09-09 12:20:47,761 INFO status has been updated to accepted
2025-09-09 12:49:10,060 INFO status has been updated to running
2025-09-09 13:07:12,900 INFO status has been updated to successful


47433ccf262f4d74a48f4e2249961e34.nc:   0%|          | 0.00/5.45M [00:00<?, ?B/s]

2025-09-09 13:07:15,785 INFO [2024-09-26T00:00:00] Watch our [Forum](https://forum.ecmwf.int/) for Announcements, news and other discussed topics.
2025-09-09 13:07:16,111 INFO Request ID is 9f676624-27c2-4262-b853-b55e25aa774c
2025-09-09 13:07:16,204 INFO status has been updated to accepted
2025-09-09 13:19:36,443 INFO status has been updated to running
2025-09-09 13:39:19,109 INFO status has been updated to successful


...

8c625de81a80087b7caa460ef64beddc.nc:   0%|          | 0.00/5.45M [00:00<?, ?B/s]

2025-09-09 14:30:29,370 INFO [2024-09-26T00:00:00] Watch our [Forum](https://forum.ecmwf.int/) for Announcements, news and other discussed topics.
2025-09-09 14:30:29,756 INFO Request ID is 0ea2f133-cf19-44f9-b1f1-99c1d6f12d05
2025-09-09 14:30:29,957 INFO status has been updated to accepted
2025-09-09 15:10:54,891 INFO status has been updated to running
2025-09-09 15:30:57,302 INFO status has been updated to successful


966b999b60ec669f46a63ceeb44718d5.nc:   0%|          | 0.00/5.47M [00:00<?, ?B/s]

2025-09-09 15:30:59,040 INFO [2024-09-26T00:00:00] Watch our [Forum](https://forum.ecmwf.int/) for Announcements, news and other discussed topics.
2025-09-09 15:30:59,410 INFO Request ID is 976b7d2e-aeb6-44b1-be20-fb86733908b6
2025-09-09 15:30:59,494 INFO status has been updated to accepted
2025-09-09 16:45:27,641 INFO status has been updated to running
2025-09-09 17:05:30,484 INFO status has been updated to successful


8d1645d0d7723d1f8670667e8513bf15.nc:   0%|          | 0.00/5.45M [00:00<?, ?B/s]

In [6]:
def download_daily_mean(year):
    req = {
        "variable": ["2m_temperature"],
        "year": [str(year)],
        "month": _month_list(),
        "day": _day_list(),
        "daily_statistic": "daily_maximum",
        "time_zone": "utc+00:00",
        "frequency": "1_hourly",
        "area": [north, west, south, 
        east],
        "format": "grib"
    }
    target = OUTDIR / f"tmean_daily_{year}.grib"
    if target.exists():
        print(f"✓ {target.name} exists - skip"); return
    cdsapi.Client().retrieve("derived-era5-land-daily-statistics", req, str(target))

for yr in range(2020, 2025): #was 1991
    download_daily_mean(yr)


2025-09-11 14:13:41,878 INFO [2024-09-26T00:00:00] Watch our [Forum](https://forum.ecmwf.int/) for Announcements, news and other discussed topics.
2025-09-11 14:13:42,244 INFO Request ID is 44a71d1a-cba5-47e7-8e91-ecad4818a25c
2025-09-11 14:13:42,321 INFO status has been updated to accepted
Recovering from connection error [('Connection aborted.', RemoteDisconnected('Remote end closed connection without response'))], attemps 1 of 500
Retrying in 120 seconds
2025-09-11 15:31:32,409 INFO status has been updated to running
2025-09-11 15:47:35,012 INFO status has been updated to successful


fbb9404c27283cda24ec3d4499db9d9f.nc:   0%|          | 0.00/5.45M [00:00<?, ?B/s]

2025-09-11 15:47:36,911 INFO [2024-09-26T00:00:00] Watch our [Forum](https://forum.ecmwf.int/) for Announcements, news and other discussed topics.
2025-09-11 15:47:37,551 INFO Request ID is 540eaa80-5ddd-4ef0-a0ab-8c7807b7a5fd
2025-09-11 15:47:37,635 INFO status has been updated to accepted
2025-09-11 16:46:03,656 INFO status has been updated to running
2025-09-11 17:02:06,037 INFO status has been updated to successful


35f1216337644b6815c0a5d682cbce04.nc:   0%|          | 0.00/5.44M [00:00<?, ?B/s]

2025-09-11 17:02:08,108 INFO [2024-09-26T00:00:00] Watch our [Forum](https://forum.ecmwf.int/) for Announcements, news and other discussed topics.
2025-09-11 17:02:08,332 INFO Request ID is fdcbdfec-bff9-4c89-9f5a-7b416bfb1bff
2025-09-11 17:02:08,419 INFO status has been updated to accepted
2025-09-11 17:53:48,662 INFO status has been updated to running
Recovering from connection error [('Connection aborted.', RemoteDisconnected('Remote end closed connection without response'))], attemps 1 of 500
Retrying in 120 seconds
Recovering from connection error [HTTPSConnectionPool(host='cds.climate.copernicus.eu', port=443): Read timed out. (read timeout=60)], attemps 1 of 500
Retrying in 120 seconds
2025-09-11 18:14:41,571 INFO status has been updated to successful


b5d914729c7780e7092ae9b0d84402df.nc:   0%|          | 0.00/5.44M [00:00<?, ?B/s]

2025-09-11 18:14:44,826 INFO [2024-09-26T00:00:00] Watch our [Forum](https://forum.ecmwf.int/) for Announcements, news and other discussed topics.
2025-09-11 18:14:45,271 INFO Request ID is c5cd7e9c-eabd-4e7e-9e54-43862e4c9773
2025-09-11 18:14:45,348 INFO status has been updated to accepted
Recovering from connection error [('Connection aborted.', RemoteDisconnected('Remote end closed connection without response'))], attemps 1 of 500
Retrying in 120 seconds
2025-09-11 18:57:09,420 INFO status has been updated to running
Recovering from connection error [HTTPSConnectionPool(host='cds.climate.copernicus.eu', port=443): Read timed out. (read timeout=60)], attemps 1 of 500
Retrying in 120 seconds
2025-09-11 19:15:13,760 INFO status has been updated to successful


c876f9a43f4a2192b45c0486ebeb3bd0.nc:   0%|          | 0.00/5.44M [00:00<?, ?B/s]

2025-09-11 19:15:15,707 INFO [2024-09-26T00:00:00] Watch our [Forum](https://forum.ecmwf.int/) for Announcements, news and other discussed topics.
2025-09-11 19:15:15,948 INFO Request ID is fe3f11a7-8c48-436d-a04e-6fda49a09374
2025-09-11 19:15:16,013 INFO status has been updated to accepted
Recovering from connection error [HTTPSConnectionPool(host='cds.climate.copernicus.eu', port=443): Read timed out. (read timeout=60)], attemps 1 of 500
Retrying in 120 seconds


KeyboardInterrupt: 

### Daily Temperature Min

In [5]:
import cdsapi, pathlib, time, concurrent.futures

# ---- edit your bounding box here ----


OUTDIR = pathlib.Path("/Volumes/Dhruv_External_Disk/climate_data/temperature/daily_data/daily_min")
OUTDIR.mkdir(parents=True, exist_ok=True)

def _day_list():
    return [f"{d:02d}" for d in range(1, 32)]
def _month_list():
    return [f"{m:02d}" for m in range(1, 13)]
def _hour_list():
    return [f"{h:02d}:00" for h in range(24)]


In [ ]:
def download_daily_mean(year):
    req = {
        "variable": ["2m_temperature"],
        "year": [str(year)],
        "month": _month_list(),
        "day": _day_list(),
        "daily_statistic": "daily_minimum",
        "time_zone": "utc+00:00",
        "frequency": "1_hourly",
        "area": [north, west, south, 
        east],
        "format": "grib"
    }
    target = OUTDIR / f"tmin_daily_{year}.grib"
    if target.exists():
        print(f"✓ {target.name} exists - skip"); return
    cdsapi.Client().retrieve("derived-era5-land-daily-statistics", req, str(target))

for yr in range(1972, 1980): #was 1991
    download_daily_mean(yr)


2025-09-18 11:34:38,276 INFO [2025-09-03T00:00:00] To improve our C3S service, we need to hear from you! Please complete this very short [survey](https://confluence.ecmwf.int/x/E7uBEQ/). Thank you.
2025-09-18 11:34:38,277 INFO [2024-09-26T00:00:00] Watch our [Forum](https://forum.ecmwf.int/) for Announcements, news and other discussed topics.
2025-09-18 11:34:38,571 INFO Request ID is 6399ed1b-f0cd-43ea-b73f-4aeb835dacfe
2025-09-18 11:34:38,645 INFO status has been updated to accepted
2025-09-18 11:34:47,101 INFO status has been updated to running
2025-09-18 11:57:01,040 INFO status has been updated to successful


8ad9065f68c274c8b46eff50f92a8137.nc:   0%|          | 0.00/5.48M [00:00<?, ?B/s]

2025-09-18 11:57:03,181 INFO [2025-09-03T00:00:00] To improve our C3S service, we need to hear from you! Please complete this very short [survey](https://confluence.ecmwf.int/x/E7uBEQ/). Thank you.
2025-09-18 11:57:03,182 INFO [2024-09-26T00:00:00] Watch our [Forum](https://forum.ecmwf.int/) for Announcements, news and other discussed topics.
2025-09-18 11:57:03,614 INFO Request ID is 9236297b-80e3-4f2c-a8f3-fb7afdf3633e
2025-09-18 11:57:03,693 INFO status has been updated to accepted
Recovering from HTTP error [500 Internal Server Error], attemps 1 of 500
Retrying in 120 seconds
Recovering from HTTP error [500 Internal Server Error], attemps 1 of 500
Retrying in 120 seconds
Recovering from HTTP error [500 Internal Server Error], attemps 2 of 500
Retrying in 120 seconds
Recovering from connection error [('Connection aborted.', RemoteDisconnected('Remote end closed connection without response'))], attemps 1 of 500
Retrying in 120 seconds
2025-09-18 13:19:40,073 INFO status has been upda

151e52cdca165892d85e7fd2e82fa4a9.nc:   0%|          | 0.00/5.49M [00:00<?, ?B/s]

2025-09-18 13:36:54,685 INFO [2025-09-03T00:00:00] To improve our C3S service, we need to hear from you! Please complete this very short [survey](https://confluence.ecmwf.int/x/E7uBEQ/). Thank you.
2025-09-18 13:36:54,686 INFO [2024-09-26T00:00:00] Watch our [Forum](https://forum.ecmwf.int/) for Announcements, news and other discussed topics.
2025-09-18 13:36:54,934 INFO Request ID is 2c094c1b-6b51-40a6-b136-1e208014604b
2025-09-18 13:36:55,180 INFO status has been updated to accepted
Recovering from HTTP error [500 Internal Server Error], attemps 1 of 500
Retrying in 120 seconds
2025-09-18 14:46:18,472 INFO status has been updated to running
Recovering from HTTP error [500 Internal Server Error], attemps 1 of 500
Retrying in 120 seconds
Recovering from HTTP error [500 Internal Server Error], attemps 1 of 500
Retrying in 120 seconds
2025-09-18 15:04:28,370 INFO status has been updated to successful


1f703b936afd7c99b49d05ad64d06381.nc:   0%|          | 0.00/5.48M [00:00<?, ?B/s]

2025-09-18 15:04:30,736 INFO [2025-09-03T00:00:00] To improve our C3S service, we need to hear from you! Please complete this very short [survey](https://confluence.ecmwf.int/x/E7uBEQ/). Thank you.
2025-09-18 15:04:30,737 INFO [2024-09-26T00:00:00] Watch our [Forum](https://forum.ecmwf.int/) for Announcements, news and other discussed topics.
2025-09-18 15:04:31,371 INFO Request ID is c98e7027-9dbc-4f27-8181-cc9cee7bba38
2025-09-18 15:04:31,563 INFO status has been updated to accepted
Recovering from HTTP error [500 Internal Server Error], attemps 1 of 500
Retrying in 120 seconds
Recovering from HTTP error [500 Internal Server Error], attemps 2 of 500
Retrying in 120 seconds
Recovering from HTTP error [500 Internal Server Error], attemps 1 of 500
Retrying in 120 seconds
Recovering from HTTP error [500 Internal Server Error], attemps 1 of 500
Retrying in 120 seconds
2025-09-18 16:39:47,571 INFO status has been updated to running
2025-09-18 17:08:07,209 INFO status has been updated to su

481377163f06766147991b21203c57b8.nc:   0%|          | 0.00/5.48M [00:00<?, ?B/s]

2025-09-18 17:08:09,591 INFO [2025-09-03T00:00:00] To improve our C3S service, we need to hear from you! Please complete this very short [survey](https://confluence.ecmwf.int/x/E7uBEQ/). Thank you.
2025-09-18 17:08:09,592 INFO [2024-09-26T00:00:00] Watch our [Forum](https://forum.ecmwf.int/) for Announcements, news and other discussed topics.
2025-09-18 17:08:09,805 INFO Request ID is 8b8952b0-729a-48e3-ad17-79ad8e333fa6
2025-09-18 17:08:10,034 INFO status has been updated to accepted
Recovering from connection error [('Connection aborted.', RemoteDisconnected('Remote end closed connection without response'))], attemps 1 of 500
Retrying in 120 seconds


KeyboardInterrupt: 

In [ ]:
def download_daily_mean(year):
    req = {
        "variable": ["2m_temperature"],
        "year": [str(year)],
        "month": _month_list(),
        "day": _day_list(),
        "daily_statistic": "daily_minimum",
        "time_zone": "utc+00:00",
        "frequency": "1_hourly",
        "area": [north, west, south, 
        east],
        "format": "grib"
    }
    target = OUTDIR / f"tmin_daily_{year}.grib"
    if target.exists():
        print(f"✓ {target.name} exists - skip"); return
    cdsapi.Client().retrieve("derived-era5-land-daily-statistics", req, str(target))

for yr in range(1980, 2010): #was 1991
    download_daily_mean(yr)


2025-09-09 12:00:24,615 INFO [2024-09-26T00:00:00] Watch our [Forum](https://forum.ecmwf.int/) for Announcements, news and other discussed topics.
2025-09-09 12:00:24,834 INFO Request ID is de365a12-091b-489a-acbe-9523903e4a10
2025-09-09 12:00:24,909 INFO status has been updated to accepted
2025-09-09 12:00:38,634 INFO status has been updated to running
2025-09-09 12:20:46,186 INFO status has been updated to successful


d4116d387399772111f5c59ff51403f2.nc:   0%|          | 0.00/5.46M [00:00<?, ?B/s]

2025-09-09 12:20:47,458 INFO [2024-09-26T00:00:00] Watch our [Forum](https://forum.ecmwf.int/) for Announcements, news and other discussed topics.
2025-09-09 12:20:47,695 INFO Request ID is eaeeb1fa-1aa1-4db5-a372-5387452b32b6
2025-09-09 12:20:47,761 INFO status has been updated to accepted
2025-09-09 12:49:10,060 INFO status has been updated to running
2025-09-09 13:07:12,900 INFO status has been updated to successful


47433ccf262f4d74a48f4e2249961e34.nc:   0%|          | 0.00/5.45M [00:00<?, ?B/s]

2025-09-09 13:07:15,785 INFO [2024-09-26T00:00:00] Watch our [Forum](https://forum.ecmwf.int/) for Announcements, news and other discussed topics.
2025-09-09 13:07:16,111 INFO Request ID is 9f676624-27c2-4262-b853-b55e25aa774c
2025-09-09 13:07:16,204 INFO status has been updated to accepted
2025-09-09 13:19:36,443 INFO status has been updated to running
2025-09-09 13:39:19,109 INFO status has been updated to successful


...

8c625de81a80087b7caa460ef64beddc.nc:   0%|          | 0.00/5.45M [00:00<?, ?B/s]

2025-09-09 14:30:29,370 INFO [2024-09-26T00:00:00] Watch our [Forum](https://forum.ecmwf.int/) for Announcements, news and other discussed topics.
2025-09-09 14:30:29,756 INFO Request ID is 0ea2f133-cf19-44f9-b1f1-99c1d6f12d05
2025-09-09 14:30:29,957 INFO status has been updated to accepted
2025-09-09 15:10:54,891 INFO status has been updated to running
2025-09-09 15:30:57,302 INFO status has been updated to successful


966b999b60ec669f46a63ceeb44718d5.nc:   0%|          | 0.00/5.47M [00:00<?, ?B/s]

2025-09-09 15:30:59,040 INFO [2024-09-26T00:00:00] Watch our [Forum](https://forum.ecmwf.int/) for Announcements, news and other discussed topics.
2025-09-09 15:30:59,410 INFO Request ID is 976b7d2e-aeb6-44b1-be20-fb86733908b6
2025-09-09 15:30:59,494 INFO status has been updated to accepted
2025-09-09 16:45:27,641 INFO status has been updated to running
2025-09-09 17:05:30,484 INFO status has been updated to successful


8d1645d0d7723d1f8670667e8513bf15.nc:   0%|          | 0.00/5.45M [00:00<?, ?B/s]

In [ ]:
def download_daily_mean(year):
    req = {
        "variable": ["2m_temperature"],
        "year": [str(year)],
        "month": _month_list(),
        "day": _day_list(),
        "daily_statistic": "daily_minimum",
        "time_zone": "utc+00:00",
        "frequency": "1_hourly",
        "area": [north, west, south, 
        east],
        "format": "grib"
    }
    target = OUTDIR / f"tmin_daily_{year}.grib"
    if target.exists():
        print(f"✓ {target.name} exists - skip"); return
    cdsapi.Client().retrieve("derived-era5-land-daily-statistics", req, str(target))

for yr in range(2010, 2024): #was 1991
    download_daily_mean(yr)


2025-09-11 14:13:41,878 INFO [2024-09-26T00:00:00] Watch our [Forum](https://forum.ecmwf.int/) for Announcements, news and other discussed topics.
2025-09-11 14:13:42,244 INFO Request ID is 44a71d1a-cba5-47e7-8e91-ecad4818a25c
2025-09-11 14:13:42,321 INFO status has been updated to accepted
Recovering from connection error [('Connection aborted.', RemoteDisconnected('Remote end closed connection without response'))], attemps 1 of 500
Retrying in 120 seconds
2025-09-11 15:31:32,409 INFO status has been updated to running
2025-09-11 15:47:35,012 INFO status has been updated to successful


fbb9404c27283cda24ec3d4499db9d9f.nc:   0%|          | 0.00/5.45M [00:00<?, ?B/s]

2025-09-11 15:47:36,911 INFO [2024-09-26T00:00:00] Watch our [Forum](https://forum.ecmwf.int/) for Announcements, news and other discussed topics.
2025-09-11 15:47:37,551 INFO Request ID is 540eaa80-5ddd-4ef0-a0ab-8c7807b7a5fd
2025-09-11 15:47:37,635 INFO status has been updated to accepted
2025-09-11 16:46:03,656 INFO status has been updated to running
2025-09-11 17:02:06,037 INFO status has been updated to successful


35f1216337644b6815c0a5d682cbce04.nc:   0%|          | 0.00/5.44M [00:00<?, ?B/s]

2025-09-11 17:02:08,108 INFO [2024-09-26T00:00:00] Watch our [Forum](https://forum.ecmwf.int/) for Announcements, news and other discussed topics.
2025-09-11 17:02:08,332 INFO Request ID is fdcbdfec-bff9-4c89-9f5a-7b416bfb1bff
2025-09-11 17:02:08,419 INFO status has been updated to accepted
2025-09-11 17:53:48,662 INFO status has been updated to running
Recovering from connection error [('Connection aborted.', RemoteDisconnected('Remote end closed connection without response'))], attemps 1 of 500
Retrying in 120 seconds
Recovering from connection error [HTTPSConnectionPool(host='cds.climate.copernicus.eu', port=443): Read timed out. (read timeout=60)], attemps 1 of 500
Retrying in 120 seconds
2025-09-11 18:14:41,571 INFO status has been updated to successful


b5d914729c7780e7092ae9b0d84402df.nc:   0%|          | 0.00/5.44M [00:00<?, ?B/s]

2025-09-11 18:14:44,826 INFO [2024-09-26T00:00:00] Watch our [Forum](https://forum.ecmwf.int/) for Announcements, news and other discussed topics.
2025-09-11 18:14:45,271 INFO Request ID is c5cd7e9c-eabd-4e7e-9e54-43862e4c9773
2025-09-11 18:14:45,348 INFO status has been updated to accepted
Recovering from connection error [('Connection aborted.', RemoteDisconnected('Remote end closed connection without response'))], attemps 1 of 500
Retrying in 120 seconds
2025-09-11 18:57:09,420 INFO status has been updated to running
Recovering from connection error [HTTPSConnectionPool(host='cds.climate.copernicus.eu', port=443): Read timed out. (read timeout=60)], attemps 1 of 500
Retrying in 120 seconds
2025-09-11 19:15:13,760 INFO status has been updated to successful


c876f9a43f4a2192b45c0486ebeb3bd0.nc:   0%|          | 0.00/5.44M [00:00<?, ?B/s]

2025-09-11 19:15:15,707 INFO [2024-09-26T00:00:00] Watch our [Forum](https://forum.ecmwf.int/) for Announcements, news and other discussed topics.
2025-09-11 19:15:15,948 INFO Request ID is fe3f11a7-8c48-436d-a04e-6fda49a09374
2025-09-11 19:15:16,013 INFO status has been updated to accepted
Recovering from connection error [HTTPSConnectionPool(host='cds.climate.copernicus.eu', port=443): Read timed out. (read timeout=60)], attemps 1 of 500
Retrying in 120 seconds


KeyboardInterrupt: 

### Hourly Precipitation  

In [5]:
import cdsapi, pathlib, time, concurrent.futures

# ---- edit your bounding box here ----


OUTDIR = pathlib.Path("/Volumes/Dhruv_External_Disk/climate_data/precipitation/hourly")
OUTDIR.mkdir(parents=True, exist_ok=True)

def _day_list():
    return [f"{d:02d}" for d in range(1, 32)]
def _month_list():
    return [f"{m:02d}" for m in range(1, 13)]
def _hour_list():
    return [f"{h:02d}:00" for h in range(24)]


In [6]:
def download_daily_mean(year):
    req = {
        "variable": ["total_precipitation"],
        "year": [str(year)],
        "month": _month_list(),
        "day": _day_list(),
        "time": [
            "00:00", "01:00", "02:00",
            "03:00", "04:00", "05:00",
            "06:00", "07:00", "08:00",
            "09:00", "10:00", "11:00",
            "12:00", "13:00", "14:00",
            "15:00", "16:00", "17:00",
            "18:00", "19:00", "20:00",
            "21:00", "22:00", "23:00"
        ],
        "data_format": "grib",
        "download_format": "unarchived",
        "area": [IBERIA_BBOX['max_lat'], IBERIA_BBOX['min_lon'], IBERIA_BBOX['min_lat'], IBERIA_BBOX['max_lon']]
    }
    target = OUTDIR / f"tp_hourly_{year}.grib"
    if target.exists():
        print(f"✓ {target.name} exists - skip"); return
    cdsapi.Client().retrieve("reanalysis-era5-land", req, str(target))

for yr in range(1960, 1961): #upto 1966 done. 
    download_daily_mean(yr)


2025-07-29 12:59:37,477 INFO [2024-09-26T00:00:00] Watch our [Forum](https://forum.ecmwf.int/) for Announcements, news and other discussed topics.
2025-07-29 12:59:37,653 INFO Request ID is c0b884e7-c2f9-4d76-afb9-475b7c6fd7e1
2025-07-29 12:59:37,736 INFO status has been updated to accepted
2025-07-29 12:59:51,340 INFO status has been updated to running


KeyboardInterrupt: 

In [14]:
def download_daily_mean(year):
    req = {
        "variable": ["total_precipitation"],
        "year": [str(year)],
        "month": _month_list(),
        "day": _day_list(),
        "time": [
            "00:00", "01:00", "02:00",
            "03:00", "04:00", "05:00",
            "06:00", "07:00", "08:00",
            "09:00", "10:00", "11:00",
            "12:00", "13:00", "14:00",
            "15:00", "16:00", "17:00",
            "18:00", "19:00", "20:00",
            "21:00", "22:00", "23:00"
        ],
        "data_format": "grib",
        "download_format": "unarchived",
        "area": [IBERIA_BBOX['max_lat'], IBERIA_BBOX['min_lon'], IBERIA_BBOX['min_lat'], IBERIA_BBOX['max_lon']]
    }
    target = OUTDIR / f"tp_hourly_{year}.grib"
    if target.exists():
        print(f"✓ {target.name} exists - skip"); return
    cdsapi.Client().retrieve("reanalysis-era5-land", req, str(target))

for yr in range(1996, 2024): 
    download_daily_mean(yr)


2025-06-11 13:16:54,743 INFO [2025-06-10T00:00:00] To improve our C3S service, we need to hear from you! Please complete this very short [survey](https://confluence.ecmwf.int/x/E7uBEQ/). Thank you.
2025-06-11 13:16:54,743 INFO [2024-09-26T00:00:00] Watch our [Forum](https://forum.ecmwf.int/) for Announcements, news and other discussed topics.
2025-06-11 13:16:55,034 INFO Request ID is 3dba523e-e75e-4c2d-8cc0-586c0503439f
2025-06-11 13:16:55,234 INFO status has been updated to accepted
2025-06-11 13:17:16,546 INFO status has been updated to running
2025-06-11 13:45:19,144 INFO status has been updated to successful


f6836341ce8de21f31eaab1feb576f25.grib:   0%|          | 0.00/194M [00:00<?, ?B/s]

2025-06-11 13:45:51,547 INFO [2025-06-10T00:00:00] To improve our C3S service, we need to hear from you! Please complete this very short [survey](https://confluence.ecmwf.int/x/E7uBEQ/). Thank you.
2025-06-11 13:45:51,549 INFO [2024-09-26T00:00:00] Watch our [Forum](https://forum.ecmwf.int/) for Announcements, news and other discussed topics.
2025-06-11 13:45:51,830 INFO Request ID is 70a45097-fe26-4f94-b6ba-6aa0afd1c782
2025-06-11 13:45:51,935 INFO status has been updated to accepted
2025-06-11 13:46:13,523 INFO status has been updated to running
2025-06-11 14:18:16,082 INFO status has been updated to successful


f5f4cd28cc38c94fa1ecd804be1a242a.grib:   0%|          | 0.00/193M [00:00<?, ?B/s]

2025-06-11 14:19:37,614 INFO [2025-06-10T00:00:00] To improve our C3S service, we need to hear from you! Please complete this very short [survey](https://confluence.ecmwf.int/x/E7uBEQ/). Thank you.
2025-06-11 14:19:37,615 INFO [2024-09-26T00:00:00] Watch our [Forum](https://forum.ecmwf.int/) for Announcements, news and other discussed topics.
2025-06-11 14:19:37,999 INFO Request ID is 86e91489-a1cd-430e-a179-9347f02d2106
2025-06-11 14:19:38,091 INFO status has been updated to accepted
2025-06-11 14:19:51,772 INFO status has been updated to running
2025-06-11 14:50:06,428 INFO status has been updated to successful


...

dadfb3ed7a2792755287995f8dcadf7a.grib:   0%|          | 0.00/193M [00:00<?, ?B/s]

2025-06-12 04:19:21,387 INFO [2025-06-10T00:00:00] To improve our C3S service, we need to hear from you! Please complete this very short [survey](https://confluence.ecmwf.int/x/E7uBEQ/). Thank you.
2025-06-12 04:19:21,388 INFO [2024-09-26T00:00:00] Watch our [Forum](https://forum.ecmwf.int/) for Announcements, news and other discussed topics.
2025-06-12 04:19:21,772 INFO Request ID is fbf91c35-cbfe-4cbf-a776-aea7b08f39c6
2025-06-12 04:19:21,882 INFO status has been updated to accepted
Recovering from connection error [('Connection aborted.', RemoteDisconnected('Remote end closed connection without response'))], attemps 1 of 500
Retrying in 120 seconds
2025-06-12 04:48:11,597 INFO status has been updated to successful


c26f28cb6b19ec4c3efa7bbc18337608.grib:   0%|          | 0.00/193M [00:00<?, ?B/s]

2025-06-12 04:48:41,634 INFO [2025-06-10T00:00:00] To improve our C3S service, we need to hear from you! Please complete this very short [survey](https://confluence.ecmwf.int/x/E7uBEQ/). Thank you.
2025-06-12 04:48:41,636 INFO [2024-09-26T00:00:00] Watch our [Forum](https://forum.ecmwf.int/) for Announcements, news and other discussed topics.
2025-06-12 04:48:41,828 INFO Request ID is 6c340d2e-9426-466a-bc7e-1ce0140f36a3
2025-06-12 04:48:41,936 INFO status has been updated to accepted
2025-06-12 04:49:32,076 INFO status has been updated to running
Recovering from connection error [('Connection aborted.', RemoteDisconnected('Remote end closed connection without response'))], attemps 1 of 500
Retrying in 120 seconds
2025-06-12 05:21:53,005 INFO status has been updated to successful


672f2312b9d5cf546835620079b4d23d.grib:   0%|          | 0.00/193M [00:00<?, ?B/s]

In [15]:
def download_daily_mean(year):
    req = {
        "variable": ["total_precipitation"],
        "year": ["1995"],
        "month": _month_list(),
        "day": _day_list(),
        "time": [
            "00:00", "01:00", "02:00",
            "03:00", "04:00", "05:00",
            "06:00", "07:00", "08:00",
            "09:00", "10:00", "11:00",
            "12:00", "13:00", "14:00",
            "15:00", "16:00", "17:00",
            "18:00", "19:00", "20:00",
            "21:00", "22:00", "23:00"
        ],
        "data_format": "grib",
        "download_format": "unarchived",
        "area": [IBERIA_BBOX['max_lat'], IBERIA_BBOX['min_lon'], IBERIA_BBOX['min_lat'], IBERIA_BBOX['max_lon']]
    }
    target = OUTDIR / f"tp_hourly_{year}.grib"
    # if target.exists():
    #     print(f"✓ {target.name} exists - skip"); return
    cdsapi.Client().retrieve("reanalysis-era5-land", req, str(target))

download_daily_mean(1995)


2025-06-16 16:10:17,493 INFO [2025-06-16T00:00:00] CC-BY licence to replace Licence to use Copernicus Products on 02 July 2025. More information available [here]([https://status.ecmwf.int](https://forum.ecmwf.int/t/cc-by-licence-to-replace-licence-to-use-copernicus-products-on-02-july-2025/13464)/)
2025-06-16 16:10:17,493 INFO [2025-06-10T00:00:00] To improve our C3S service, we need to hear from you! Please complete this very short [survey](https://confluence.ecmwf.int/x/E7uBEQ/). Thank you.
2025-06-16 16:10:17,493 INFO [2024-09-26T00:00:00] Watch our [Forum](https://forum.ecmwf.int/) for Announcements, news and other discussed topics.
2025-06-16 16:10:17,928 INFO Request ID is 1bac4286-bd2a-4851-b460-cfcbc5102ef4
2025-06-16 16:10:18,031 INFO status has been updated to accepted
2025-06-16 16:10:31,640 INFO status has been updated to running
2025-06-16 16:42:40,910 INFO status has been updated to successful


e8865e36b47fe91974d3a5da317478f.grib:   0%|          | 0.00/197M [00:00<?, ?B/s]

### Monthly Data

#### Rain

In [15]:
OUTDIR = pathlib.Path("/Volumes/Dhruv_External_Disk/climate_data/precipitation/monthly")
OUTDIR.mkdir(parents=True, exist_ok=True)

dataset = "reanalysis-era5-land-monthly-means"
request = {
    "variable": [
        "total_precipitation"
    ],
    "year": [
        "1950", "1951", "1952",
        "1953", "1954", "1955",
        "1956", "1957", "1958",
        "1959", "1960", "1961",
        "1962", "1963", "1964",
        "1965", "1966", "1967",
        "1968", "1969", "1970",
        "1971", "1972", "1973",
        "1974", "1975", "1976",
        "1977", "1978", "1979",
        "1980", "1981", "1982",
        "1983", "1984", "1985",
        "1986", "1987", "1988",
        "1989", "1990", "1991",
        "1992", "1993", "1994",
        "1995", "1996", "1997",
        "1998", "1999", "2000",
        "2001", "2002", "2003",
        "2004", "2005", "2006",
        "2007", "2008", "2009",
        "2010", "2011", "2012",
        "2013", "2014", "2015",
        "2016", "2017", "2018",
        "2019", "2020", "2021",
        "2022", "2023", "2024"
    ],
    "month": [
        "01", "02", "03",
        "04", "05", "06",
        "07", "08", "09",
        "10", "11", "12"
    ],
    "time": [
        "00:00", "01:00", "02:00",
        "03:00", "04:00", "05:00",
        "06:00", "07:00", "08:00",
        "09:00", "10:00", "11:00",
        "12:00", "13:00", "14:00",
        "15:00", "16:00", "17:00",
        "18:00", "19:00", "20:00",
        "21:00", "22:00", "23:00"
    ],
    "data_format": "grib",
    "download_format": "unarchived",
        "area": [IBERIA_BBOX['max_lat'], IBERIA_BBOX['min_lon'], IBERIA_BBOX['min_lat'], IBERIA_BBOX['max_lon']]
}
target = OUTDIR / f"tp_monthly_average.grib"

client = cdsapi.Client()
client.retrieve(dataset, request,str(target))


2025-06-12 05:22:25,264 INFO [2025-06-10T00:00:00] To improve our C3S service, we need to hear from you! Please complete this very short [survey](https://confluence.ecmwf.int/x/E7uBEQ/). Thank you.
2025-06-12 05:22:25,265 INFO [2024-09-26T00:00:00] Watch our [Forum](https://forum.ecmwf.int/) for Announcements, news and other discussed topics.
2025-06-12 05:22:25,916 INFO Request ID is fa4fe887-45fd-4ea8-bfcc-e19e6a0d972d
2025-06-12 05:22:25,995 INFO status has been updated to accepted
Recovering from connection error [('Connection aborted.', RemoteDisconnected('Remote end closed connection without response'))], attemps 1 of 500
Retrying in 120 seconds
2025-06-12 05:51:56,372 INFO status has been updated to running
Recovering from connection error [('Connection aborted.', RemoteDisconnected('Remote end closed connection without response'))], attemps 1 of 500
Retrying in 120 seconds
Recovering from connection error [('Connection aborted.', RemoteDisconnected('Remote end closed connection

d1d4b0173ccfcf95c537feba0adee442.grib:   0%|          | 0.00/974M [00:00<?, ?B/s]

ChunkedEncodingError: ('Connection broken: IncompleteRead(43574656 bytes read, 977228996 more expected)', IncompleteRead(43574656 bytes read, 977228996 more expected))

#### Temeperature

In [16]:
OUTDIR = pathlib.Path("/Volumes/Dhruv_External_Disk/climate_data/temperature/monthly_data")
OUTDIR.mkdir(parents=True, exist_ok=True)

dataset = "reanalysis-era5-land-monthly-means"
request = {
    "variable": [
        "total_precipitation"
    ],
    "year": [
        "1950", "1951", "1952",
        "1953", "1954", "1955",
        "1956", "1957", "1958",
        "1959", "1960", "1961",
        "1962", "1963", "1964",
        "1965", "1966", "1967",
        "1968", "1969", "1970",
        "1971", "1972", "1973",
        "1974", "1975", "1976",
        "1977", "1978", "1979",
        "1980", "1981", "1982",
        "1983", "1984", "1985",
        "1986", "1987", "1988",
        "1989", "1990", "1991",
        "1992", "1993", "1994",
        "1995", "1996", "1997",
        "1998", "1999", "2000",
        "2001", "2002", "2003",
        "2004", "2005", "2006",
        "2007", "2008", "2009",
        "2010", "2011", "2012",
        "2013", "2014", "2015",
        "2016", "2017", "2018",
        "2019", "2020", "2021",
        "2022", "2023", "2024"
    ],
    "month": [
        "01", "02", "03",
        "04", "05", "06",
        "07", "08", "09",
        "10", "11", "12"
    ],
    "time": [
        "00:00", "01:00", "02:00",
        "03:00", "04:00", "05:00",
        "06:00", "07:00", "08:00",
        "09:00", "10:00", "11:00",
        "12:00", "13:00", "14:00",
        "15:00", "16:00", "17:00",
        "18:00", "19:00", "20:00",
        "21:00", "22:00", "23:00"
    ],
    "data_format": "grib",
    "download_format": "unarchived",
        "area": [IBERIA_BBOX['max_lat'], IBERIA_BBOX['min_lon'], IBERIA_BBOX['min_lat'], IBERIA_BBOX['max_lon']]
}
target = OUTDIR / f"t2m_monthly_average.grib"

client = cdsapi.Client()
client.retrieve(dataset, request,str(target))


2025-06-12 13:16:09,016 INFO [2025-06-10T00:00:00] To improve our C3S service, we need to hear from you! Please complete this very short [survey](https://confluence.ecmwf.int/x/E7uBEQ/). Thank you.
2025-06-12 13:16:09,016 INFO [2024-09-26T00:00:00] Watch our [Forum](https://forum.ecmwf.int/) for Announcements, news and other discussed topics.
2025-06-12 13:16:11,985 INFO Request ID is 7dc161a5-94ac-423d-8ac9-f3d96ec90f44
2025-06-12 13:16:12,442 INFO status has been updated to accepted
2025-06-12 14:26:50,859 INFO status has been updated to successful


d1d4b0173ccfcf95c537feba0adee442.grib:   0%|          | 0.00/974M [00:00<?, ?B/s]

'/Volumes/Dhruv_External_Disk/climate_data/temperature/monthly_data/t2m_monthly_average.grib'